In [10]:
import codecs
import nltk.data
import pymorphy2
import re
import os
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from itertools import takewhile
morph = pymorphy2.MorphAnalyzer()

In [29]:
delims = ",.!?/&:;@'*()«»...\n\t\`"
stopwords = open("stopwords.txt", 'r').read().split("\n")
res = []

# Delete delimiters, remove stopwords, make float . stay as _
def process(text):
    title = text.split('\n')[0]
    text = re.sub(r'\[.+?\]\s*', ' ', text)
    text = re.sub(r'«.+?»\s*', ' ', text)
    text = re.sub(r'-', ' ', text)
    text = re.sub(r' — ', ' . ', text)
    text = re.sub(r'([+-]?[0-9]+)\.([0-9]+)',r'\1_\2', text)
    text = re.sub(r'([+-]?[0-9]+),([0-9]+)',r'\1_\2', text)
    stopwords_re = re.compile('|'.join(["\W" + w + "\W" for w in stopwords]))
    text = stopwords_re.sub(".", text.lower())
    text = stopwords_re.sub(".", text)
    lines = [w.strip() for w in re.split("["+"\\".join(delims)+"]", text) if w != ""]
    return title, [' '.join(filter(lambda x: re.match(r'^-?[А-Яа-я]+|-?[A-Za-z]+|-?[1-9]+$', x), 
                                   text.split())) for text in lines]
 
# if there is n-gram keyword with high certanity, delete related n-1-grams
def remove_dups(less, more, num):
    words = [x[0] for x in less]
    todel = []
    for s in more:
        l = s[0].split(' ')
        cnt = 0
        for i in range(len(l) - num + 1):
            if not " ".join(l[i: i + num]).strip() in words:
                break
            cnt = cnt + 1
        if cnt == 2:
            todel += [" ".join(l[i: i + num]) for i in range(len(l) - num + 1)]
    return list(filter(lambda s: s[0] not in todel, less))

# check if difference is only in wordforms
def are_alike(s_a, s_b):
    a = s_a.split(' ')
    b = s_b.split(' ')
    for i in range(len(a)):
        if morph.parse(a[i])[0].normal_form != morph.parse(b[i])[0].normal_form:
            return False
    return True

# remove keywords that differ in wordform only
def remove_wordforms(l):
    new_l = []
    for a in range(len(l)):
        flag = True
        for b in range(a):
            if a != b:
                if are_alike(l[a][0], l[b][0]):
                    flag = False
                    break
        if flag:
            new_l.append(l[a])
    return new_l

def get_confidence(ngram_freq, nless1_freq):
    return (ngram_freq * (ngram_freq * 0.1)) / float(nless1_freq) if ngram_freq > 1 and nless1_freq > 1 else 0

In [30]:
t = [process(codecs.open("habr/" + f, 'r', "utf_8_sig").read())
        for f in os.listdir("habr")
        if f.endswith(".txt")]

In [31]:
def get_keywords(data, n1 = 5, n2 = 15, n3 = 10, n4 = 5, n5 = 5, b1 = 0.01):
    total = 0.0
    monograms = []
    bigrams = []
    trigrams = []
    fourgrams = []
    fivegrams = []
    # get ngrams
    for line in data:
        if len(line) == 0:
            continue
        #print(line.split(" "))
        line_len = len(line.split(" "))
        total += line_len

        token = nltk.word_tokenize(line)
        monograms += ngrams(token, 1)
        if line_len < 2:
            continue
        bigrams += ngrams(token, 2)
        if line_len < 3:
            continue
        trigrams += ngrams(token, 3)
        if line_len < 4:
            continue
        fourgrams += ngrams(token, 4)
        if line_len < 5:
            continue
        fivegrams += ngrams(token, 5)

    # get frequencies
    d1 = Counter(monograms)
    d2 = Counter(bigrams)
    d3 = Counter(trigrams)
    d4 = Counter(fourgrams)
    d5 = Counter(fivegrams)

    # get probabilities of occurence of ngram in text, choose keywords based on probability and frequence
    key1 = sorted([(" ".join(x), float(d1[x] / total)) 
            for x in d1], key=lambda tup: tup[1], reverse=True)[:n1]
    # if float(d1[x] / total) > b1]
    key2 = sorted([(" ".join(x), get_confidence(d2[x], d1[(x[0],)])) 
            for x in d2], key=lambda tup: tup[1], reverse=True)[:n2]
    key1 = remove_dups(key1, key2, 1)
    
    key3 = sorted([(" ".join(x), get_confidence(d3[x], d2[(x[0], x[1],)])) 
            for x in d3], key=lambda tup: tup[1], reverse=True)[:n3]
    key2 = remove_dups(key2, key3, 2)

    key4 = sorted([(" ".join(x), get_confidence(d4[x], d3[(x[0], x[1], x[2],)])) 
            for x in d4], key=lambda tup: tup[1], reverse=True)[:n4]
    key3 = remove_dups(key3, key4, 3)    
    
    key5 = sorted([(" ".join(x),get_confidence(d5[x], d4[(x[0], x[1], x[2], x[3],)])) 
            for x in d5], key=lambda tup: tup[1], reverse=True)[:n5]
    key4 = remove_dups(key4, key5, 4)
    #[x[0] for x in ]
    key1 = remove_wordforms(key1)
    key2 = remove_wordforms(key2)
    key3 = remove_wordforms(key3)
    key4 = remove_wordforms(key4)
    key5 = remove_wordforms(key5)
    keywords = key1 + key2 + key3 + key4 + key5
    print(key1)
    print(key2)
    print(key3)
    return keywords

In [32]:
get_keywords(t[0][1], n1 = 3, n2 = 20, n3 = 20, n4 = 1, n5 = 1, b1 = 0.1)

[('node', 0.0365296803652968), ('http', 0.015220700152207), ('js', 0.015220700152207)]
[('входящих запросов', 0.2), ('event loop', 0.2), ('событийная модель', 0.2), ('выполнять работу', 0.2), ('одной причине', 0.2), ('ввода вывода', 0.2), ('давайте начнём', 0.13333333333333333), ('перед node', 0.13333333333333333), ('веб сервер', 0.1285714285714286), ('http сервер', 0.09000000000000001), ('n 2', 0.06666666666666667), ('веб приложения', 0.05714285714285715), ('крах масштабируемости', 0), ('насколько асинхронна', 0), ('решишься подойти', 0), ('прочитал полный', 0), ('вызов cgi', 0), ('производительностью cpu', 0), ('система отблагодарит', 0), ('дочерние процессы', 0)]
[('запускаете приложение прямо', 0), ('веб сервер перед', 0), ('умеющих думать самостоятельно', 0), ('unix слишком сложен', 0), ('javascript http сервер', 0), ('c 5 http', 0), ('современные http серверы', 0), ('могут разрабатывать быстрые', 0), ('учитывая оригинальную маркетинговую', 0), ('адекватный программист заметит', 0

[('node', 0.0365296803652968),
 ('http', 0.015220700152207),
 ('js', 0.015220700152207),
 ('входящих запросов', 0.2),
 ('event loop', 0.2),
 ('событийная модель', 0.2),
 ('выполнять работу', 0.2),
 ('одной причине', 0.2),
 ('ввода вывода', 0.2),
 ('давайте начнём', 0.13333333333333333),
 ('перед node', 0.13333333333333333),
 ('веб сервер', 0.1285714285714286),
 ('http сервер', 0.09000000000000001),
 ('n 2', 0.06666666666666667),
 ('веб приложения', 0.05714285714285715),
 ('крах масштабируемости', 0),
 ('насколько асинхронна', 0),
 ('решишься подойти', 0),
 ('прочитал полный', 0),
 ('вызов cgi', 0),
 ('производительностью cpu', 0),
 ('система отблагодарит', 0),
 ('дочерние процессы', 0),
 ('запускаете приложение прямо', 0),
 ('веб сервер перед', 0),
 ('умеющих думать самостоятельно', 0),
 ('unix слишком сложен', 0),
 ('javascript http сервер', 0),
 ('c 5 http', 0),
 ('современные http серверы', 0),
 ('могут разрабатывать быстрые', 0),
 ('учитывая оригинальную маркетинговую', 0),
 ('адек

In [33]:
with open('out_every_doc.txt','w') as f:
    for (title, data) in t:    
        f.write(title + "\n")
        print(title)
        keywords = get_keywords(data)
        for string in keywords:
            if string[1] > 0:
                res.append(string)
            f.write(str(string) + ", ")
        f.write("\n\n")
        print("\n\n")

Node.js — раковая опухоль
[('node', 0.0365296803652968), ('js', 0.015220700152207)]
[('входящих запросов', 0.2), ('event loop', 0.2), ('событийная модель', 0.2), ('выполнять работу', 0.2), ('одной причине', 0.2), ('ввода вывода', 0.2), ('давайте начнём', 0.13333333333333333), ('перед node', 0.13333333333333333), ('веб сервер', 0.1285714285714286), ('http сервер', 0.09000000000000001), ('n 2', 0.06666666666666667), ('веб приложения', 0.05714285714285715), ('крах масштабируемости', 0), ('насколько асинхронна', 0), ('решишься подойти', 0)]
[('запускаете приложение прямо', 0), ('веб сервер перед', 0), ('умеющих думать самостоятельно', 0), ('unix слишком сложен', 0), ('javascript http сервер', 0), ('c 5 http', 0), ('современные http серверы', 0), ('могут разрабатывать быстрые', 0), ('учитывая оригинальную маркетинговую', 0), ('адекватный программист заметит', 0)]



История успеха русского шестнадцатилетнего юноши
[('афера', 0.010221465076660987), ('деньги', 0.008517887563884156), ('карты',

[('часть', 0.008422234699606962), ('поэтому', 0.0050533408197641775), ('семен', 0.004491858506457047), ('данных', 0.0039303761931499155), ('николай', 0.003368893879842785)]
[('дисковую группу', 0.30000000000000004), ('ленточная библиотека', 0.2), ('менеджер проекта', 0.2), ('сухим молоком', 0.2), ('запуск комплекса', 0.2), ('мою заявку', 0.13333333333333333), ('запчасть сразу', 0.13333333333333333), ('9 утра', 0.13333333333333333), ('голове моей', 0.08), ('сразу отсекли', 0), ('счастливо улыбался', 0), ('монтаж серверов', 0)]
[('начала рабочего дня', 0.2), ('перед собой заветную', 0), ('дополнительно подключился модуль', 0), ('перед столичным гостем', 0), ('имея ввиду библиотеку', 0), ('время таксист вез', 0), ('безвкусно украшенные стены', 0), ('мою бытность инженером', 0), ('затхлым запахом сырости', 0), ('появлением второй площадки', 0)]



Почему Microsoft Internet Explorer 11 — худшее, что могло случиться с нами
[('версии', 0.013605442176870748), ('ie10', 0.012244897959183673), ('

[('программистов', 0.013333333333333334), ('код', 0.012), ('часто', 0.009333333333333334)]
[('реальной жизни', 0.30000000000000004), ('психологической деформации', 0.2), ('работы программиста', 0.2), ('данный момент', 0.15000000000000002), ('должен работать', 0.1), ('проще говоря', 0), ('естественным порождением', 0), ('плавной проруткой', 0), ('программиста имеет', 0), ('своему субъективному', 0), ('момент ввиду', 0)]
[('причинно следственных связей', 0.4), ('уровень социального взаимодействия', 0.2), ('делают вбивание автокомплита', 0), ('должен работать ровно', 0), ('целую цепочку причинно', 0), ('достаточно смело говорить', 0), ('огромные потоки размышлений', 0), ('имеют указанную специфику', 0), ('легкая форма аутизма', 0), ('отдельные программисты помимо', 0)]



300 миллионов пользователей и переход на WebKit
[('webkit', 0.02237136465324385), ('стандартов', 0.011185682326621925), ('html5', 0.011185682326621925)]
[('продолжат работать', 0.2), ('проект webkit', 0.2), ('internet ex

In [34]:
with open ('out_global.txt', 'w') as f:
    for string in res:
        f.write(str(string) + ", ")